# Uniview module for SN 1987a

*Aaron Geller, 2018*

### Imports and function definitions

In [14]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data" 
import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
import uvmodlib.v1 as uvmod

### USES Conf Template 

In [63]:
Template = """mesh
{  
    data center    ./modules/$folderName/center.raw

    cullRadius $cr
    glslVersion 330
    
    propertyCollection 
    {        
        __objectName__
        {           

            vec1f SNRadMax 1e5 | public | desc "max SN radius" | widget slider | range 0 1e6
            vec1f SNTimeMax 2 | public | desc "max SN time" | widget slider | range 0 30
            vec1f eventTime 0 | public | desc "event time " | widget slider | range 0 30
            vec4f lineColor 0.0 0.25 0 0.5 | public
            
        }
    }
    ############# Optical explosion
    pass
    {
        useDataObject center
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/SN.gs
                vertexShader ./modules/$folderName/SN.vs
                fragmentShader   ./modules/$folderName/SN.fs
                texture cmap ./modules/$folderName/cmap.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                }  
                
                stateManagerVar __objectName__.SNRadMax  SNRadMax
                stateManagerVar __objectName__.SNTimeMax  SNTimeMax
                stateManagerVar __objectName__.eventTime  eventTime
                
                glState
                {
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE true
                    UV_WRITE_MASK_DEPTH false
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }


}"""

### SN class

In [64]:
class SN1987a():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("SN.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("SN.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("SN.fs",dataFolder)
        self.cr = 1000
        self.Scale = 1

    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(Template)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr,
                                 Scale = self.Scale
                                 ))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [65]:
model = SN1987a(uvmod.OrbitalObject())
generator = uvmod.Generator()
scene = uvmod.Scene()
scene.setname("SN1987a")
scene.setparent("Earth")
scene.setentrydist(100000)
scene.setunit(1)
scene.setsurfacepositionerrotation(0,0.,-90)
scene.setpositionfile(uvmod.PositionFileTypes.Surface("Earth",  0.0, -89., -0.2))

modinfo = uvmod.ModuleInformation()

### Specify Settings and generate the module

In [66]:
model.object.setcameraradius(10)
model.object.setcoord(scene.name)
model.object.setname("SN1987a")
model.object.setguiname("/KavliLecture/Larson/SN1987a")
model.object.settargetradius(20)
model.object.showatstartup(False)
model.cr = 10000
modinfo.setname("SN1987a")
modinfo.setauthor("Aaron Geller<sup>1</sup> and Shane Larson<sup>2</sup><br />(1)Adler Planetarium,br />(2)Northwestern University")
modinfo.cleardependencies()
modinfo.setdesc("Uniview module for SN 19878a")
#modinfo.setthumbnail("data/R0010133.JPG")
modinfo.setversion("1.0")

generator.generate("SN1987a",[scene],[model],modinfo)
uvmod.Utility.senduvcommand(model.object.name+".reload")

## Helper Functions for modifing code
*Reload Module and Shaders in Uniview*

In [9]:
uvmod.Utility.senduvcommand(model.object.name+".reload; system.reloadallshaders")

Unable to connect to Uniview


*Copy modified Shader files and reload*

In [31]:
from config import Settings

In [75]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+model.object.name)
uvmod.Utility.senduvcommand(model.object.name+".reload")

### Create colormap texture

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [3]:
gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

def plot_cmap(colormap):
    fig=plt.imshow(gradient, aspect=1, cmap=colormap)
    plt.axis('off')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.savefig("data/cmap.png", bbox_inches='tight',pad_inches=0)

plot_cmap('plasma')